In [217]:
import math
import numpy as np
import random as rd
import tkinter as tk
from shapely.geometry import LineString

In [218]:
FILENAME = 'test0.txt'
LINE_DIS = 40
POINT_RADIUS = 5
COLORS = ['red', 'blue', 'green']

# For tests
F_WIDTH = 10
F_HEIGHT = 10
POINTS_NUMBER = 3

# rd.seed(200)

In [219]:
# Return true if line segments AB and CD intersect
def intersect(A, B, C, D):
    if A == C or A == D or B == C or B == D:
        return False
    return LineString([A, B]).intersects(LineString([C, D]))


def read_file(filename):
    with open('./data/' + filename, 'r') as f:
        # Return array with points, max width and height of the field
        coords = [ list(map(int, x.split(';'))) for x in f.readlines() ]
        return coords, np.max(np.array(coords)[:,0]), np.max(np.array(coords)[:,1])


# coords, F_WIDTH, F_HEIGHT = read_file(FILENAME)

coords = [ [rd.randint(1, F_WIDTH), rd.randint(1, F_HEIGHT) ] for _ in range(POINTS_NUMBER) ]
coords = [ [ x * LINE_DIS, y * LINE_DIS ] for x, y in coords ]

print(coords)

[[120, 360], [40, 360], [80, 120]]


In [220]:
class GUI:
    def __init__(self, f_width, f_height):
        self.f_width = f_width
        self.f_height = f_height

        self.root = tk.Tk()
        self.root.title('Problem kolorowania mapy')
        self.root.resizable(False, False)

        self.canvas = tk.Canvas(self.root, width=(1 + f_width) * LINE_DIS, height=(1 + f_height) * LINE_DIS, bg="#fff")
        self.canvas.pack()
        self.root.update()

    def draw_circle(self, x, y, radius=POINT_RADIUS, color='#000'):
        self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius, fill=color, outline='')

    def draw_background(self):
        for i in range(1, self.f_height + 1):
            for j in range(1, self.f_width + 1):
                self.draw_circle(i * LINE_DIS, j * LINE_DIS, color='#eee')

    def draw_line(self, x1, y1, x2, y2):
        self.canvas.create_line(x1, y1, x2, y2, fill="#242424")

    def draw_color_points(self, coords, colors):
        for point, color in zip(coords, colors):
            self.draw_circle(*point, color=COLORS[color - 1])

    def draw_points(self, coords):
        for co in coords:
            self.draw_circle(*co)

In [221]:
gui = GUI(F_WIDTH, F_HEIGHT)
gui.draw_background()

In [222]:
lines = []

for _ in range(3):
    rd.shuffle(coords)
    for X in coords:
        # Sort by euclidean distance and get the first nearest
        dstn = sorted(coords, key=lambda point: math.dist(point, X))

        for Y in dstn[1:]:
            # If two points on the line have been already chosen
            if [X, Y] in lines or [Y, X] in lines: continue
            
            # Check if current line intersects any other
            if any(map(lambda x: intersect(X, Y, *x), lines)): continue

            gui.draw_line(*X, *Y)
            lines.append([X, Y])
            break

lines

[[[40, 360], [120, 360]], [[120, 360], [80, 120]], [[80, 120], [40, 360]]]

In [223]:
solutions = []


class CSP:
    def __init__(self, coords, lines):
        self.m = len(COLORS)
        self.v = len(coords)
        self.coords = [*coords]
        self.colors = [0] * self.v
        self.graph = [ [0] * self.v for _ in range(self.v) ]

        for s, e in lines:
            r = coords.index(s)
            c = coords.index(e)
            self.graph[r][c] = 1
        
    def is_complete(self) -> bool:
        return not any( self.graph[i][j] and self.colors[i] == self.colors[j] for i in range(len(self.graph)) for j in range(i + 1, len(self.graph)) )
        
    def save_solution(self) -> None:
        if self.colors not in solutions:
            solutions.append([*self.colors])

    def rbt(self, v=0) -> bool:
        if v == self.v:
            if self.is_complete():
                self.save_solution()
                return True
            return False

        for i in range(1, self.m + 1):
            self.colors[v] = i
            self.rbt(v + 1)
            self.colors[v] = 0


csp = CSP(coords, lines)
csp.rbt()
print(solutions)

colors = solutions[0]

[[1, 2, 1], [1, 2, 3], [1, 3, 1], [1, 3, 2], [2, 1, 2], [2, 1, 3], [2, 3, 1], [2, 3, 2], [3, 1, 2], [3, 1, 3], [3, 2, 1], [3, 2, 3]]


### CSP - map-coloring problem

- Variables - points (regions)
- Domains - colors (red, blue, green, etc)
- Contraints - adjacent regions must have different colors

In [224]:
gui.draw_color_points(coords, colors)
# gui.draw_points(coords)
gui.root.mainloop()